In [1]:
import zipfile
from urllib import request
import imutils
import matplotlib.image as mpimg
import random
%matplotlib inline
import cv2
from glob import glob
import keras,math
import cv2 as cv 
import pandas as pd
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Reshape ,MaxPooling2D, advanced_activations
from keras.layers import concatenate, Concatenate,Dense, Flatten,SeparableConv2D,LeakyReLU,GlobalAveragePooling2D
from keras.layers import BatchNormalization, RepeatVector, Permute, Activation, Reshape, Flatten,Multiply
from keras.layers.advanced_activations import PReLU

from keras.layers import Lambda,Add,Dropout,SpatialDropout2D,DepthwiseConv2D,normalization
from keras.models import Sequential, Model 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.backend import clear_session
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.layers import MaxPooling2D, Input, Dense, Flatten
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.gridspec as gridspec
 
import tensorflow as tf 
import keras
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
 
# aita die akta tensor e sob image k nie astasi
 
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Lambda,Subtract,concatenate,Add,merge,add
#from SpectralNormalizationKeras import DenseSN, ConvSN2D

from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Lambda,Subtract,concatenate,Add,merge,add
#from SpectralNormalizationKeras import DenseSN, ConvSN2D
 
 
from keras.optimizers import Adam 

Using TensorFlow backend.


# Dataset loading

In [2]:
x_train = np.load('./dataset/x_train.npy')
y_train = np.load('./dataset/y_train.npy')
x_test = np.load('./dataset/x_test.npy')
y_test = np.load('./dataset/y_test.npy')

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(120, 384, 384, 1) (120, 384, 384, 1)
(30, 384, 384, 1) (30, 384, 384, 1)


## In this notebook, demonstrated solution is utilizing the regression loss family for the given problem.

In [3]:
 
 
def plot_sample(lr, sr):
    plt.figure(figsize=(14, 12))

    images = [lr, sr]
    titles = ['recovered code', 'input code']

    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, 2, i+1)
        plt.imshow(img,cmap='gray')
        plt.title(title)
        plt.xticks([])
        plt.yticks([])
def imgps(x):
    p = x[0] * 255
    p = p.astype(np.uint8)
    p = Image.fromarray(p)
    return p


def psnr(y_true,y_pred):
    return tf.image.psnr(y_true,y_pred,1.0)
def ssim(y_true,y_pred):
    return tf.image.ssim(y_true,y_pred,1.0)

In [4]:
def unt2(ip,f):
  
    conv1 = Conv2D(f, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(ip)
    conv1 = Conv2D(f, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    
    conv2 = Conv2D(f*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(f*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    
    conv3 = Conv2D(f*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(f*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    
    conv4 = Conv2D(f*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(f*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    

    conv5 = Conv2D( f*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(f*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    
    u5=Conv2DTranspose(f, 3, strides=16,activation='relu', padding='same')(drop5)
    u4=Conv2DTranspose(f, 3, strides=8,activation='relu', padding='same')(pool3)
    u3=Conv2DTranspose(f, 3, strides=4,activation='relu', padding='same')(pool2)
    u2=Conv2DTranspose(f, 3, strides=2,activation='relu', padding='same')(pool1)
    u1=Conv2DTranspose(f, 3, strides=1,activation='relu', padding='same')(ip)
    
    
    col = concatenate([u1,u2,u3,u4,u5], axis = 3)
    col = Conv2D(3, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(col)
    ip = Conv2D(3, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(ip)
    col= mdsr1(col,16) 
    sm = add([col,ip])

 
    x = Conv2D(1,1,  activation ='sigmoid', padding = 'same', kernel_initializer = 'glorot_normal')(sm)
        
 
    return x
def mdsr1(ix,f):
    x=Conv2D(f, kernel_size=3, strides=1, padding='same')(ix)
    
    x1=r1(x,f)
    x1=r1(x1,f)
    x2=r1(x,f)
    x2=r1(x2,f)
    x3=r1(x,f)
    x3=r1(x3,f)
    x=add([x1,x2,x3])
    x=concatenate([x,x1,x2,x3 ], axis = 3)
    x=Conv2D(3, kernel_size=3, strides=1, padding='same')(x)
    return x 

def r1(input_tensor, features ):
    x = Conv2D(features, 3, activation='relu', padding='same')(input_tensor)
    x = Conv2D(features, 3, padding='same')(x)
    return add([input_tensor, x])


In [5]:
w=384
input_img = Input(shape=(w,w,1))

x= Conv2D(64, kernel_size=5, strides=1, padding='same')(input_img) 
 
x= mdsr1(x,16) 
x=unt2(x,16)

 
 


model = Model(input_img, x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.compile(optimizer = Adam(lr=2e-4) ,loss = 'mae', metrics=[psnr,ssim, "accuracy"])

In [7]:
 

filepath='Bwt_rgrsn.hdf5'
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             mode='min')

In [ ]:
model.fit(x_train, y_train, validation_split = .2, epochs = 70000, batch_size = 8, shuffle =  True, callbacks =  [checkpoint ])

Instructions for updating:
Use tf.cast instead.
Train on 96 samples, validate on 24 samples
Epoch 1/70000
96/96 [==============================] - 15s 156ms/step - loss: 0.4917 - psnr: 6.1251 - ssim: 0.0388 - acc: 0.5024 - val_loss: 0.4618 - val_psnr: 6.3540 - val_ssim: 0.0720 - val_acc: 0.5159

Epoch 00001: saving model to Bwt_rgrsn.hdf5
Epoch 2/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.4270 - psnr: 5.7056 - ssim: 0.1247 - acc: 0.5052 - val_loss: 0.3902 - val_psnr: 6.4133 - val_ssim: 0.2059 - val_acc: 0.5130

Epoch 00002: saving model to Bwt_rgrsn.hdf5
Epoch 3/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.3546 - psnr: 7.0407 - ssim: 0.2788 - acc: 0.6144 - val_loss: 0.3064 - val_psnr: 7.7675 - val_ssim: 0.3668 - val_acc: 0.7601

Epoch 00003: saving model to Bwt_rgrsn.hdf5
Epoch 4/70000
96/96 [==============================] - 7s 77ms/step - loss: 0.2344 - psnr: 8.8269 - ssim: 0.4609 - acc: 0.8269 - val_loss: 0.1585 - val_psnr: 9.3815 


Epoch 00032: saving model to Bwt_rgrsn.hdf5
Epoch 33/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0756 - psnr: 12.0014 - ssim: 0.7225 - acc: 0.9248 - val_loss: 0.0752 - val_psnr: 11.8264 - val_ssim: 0.7225 - val_acc: 0.9251

Epoch 00033: saving model to Bwt_rgrsn.hdf5
Epoch 34/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0754 - psnr: 12.0292 - ssim: 0.7232 - acc: 0.9250 - val_loss: 0.0743 - val_psnr: 11.9137 - val_ssim: 0.7260 - val_acc: 0.9261

Epoch 00034: saving model to Bwt_rgrsn.hdf5
Epoch 35/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0754 - psnr: 12.0551 - ssim: 0.7234 - acc: 0.9250 - val_loss: 0.0757 - val_psnr: 11.8532 - val_ssim: 0.7209 - val_acc: 0.9246

Epoch 00035: saving model to Bwt_rgrsn.hdf5
Epoch 36/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0752 - psnr: 12.1079 - ssim: 0.7248 - acc: 0.9252 - val_loss: 0.0748 - val_psnr: 11.9123 - val_ssim: 0.7239 - val_acc: 0.9255



Epoch 00065: saving model to Bwt_rgrsn.hdf5
Epoch 66/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0666 - psnr: 12.8542 - ssim: 0.7558 - acc: 0.9337 - val_loss: 0.0680 - val_psnr: 12.4199 - val_ssim: 0.7499 - val_acc: 0.9322

Epoch 00066: saving model to Bwt_rgrsn.hdf5
Epoch 67/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0670 - psnr: 12.7871 - ssim: 0.7541 - acc: 0.9332 - val_loss: 0.0657 - val_psnr: 12.5392 - val_ssim: 0.7585 - val_acc: 0.9345

Epoch 00067: saving model to Bwt_rgrsn.hdf5
Epoch 68/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0678 - psnr: 12.7954 - ssim: 0.7520 - acc: 0.9324 - val_loss: 0.0696 - val_psnr: 12.3638 - val_ssim: 0.7444 - val_acc: 0.9306

Epoch 00068: saving model to Bwt_rgrsn.hdf5
Epoch 69/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0665 - psnr: 12.8924 - ssim: 0.7561 - acc: 0.9337 - val_loss: 0.0673 - val_psnr: 12.4766 - val_ssim: 0.7526 - val_acc: 0.9329



Epoch 00098: saving model to Bwt_rgrsn.hdf5
Epoch 99/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0577 - psnr: 13.6472 - ssim: 0.7876 - acc: 0.9425 - val_loss: 0.0639 - val_psnr: 12.8133 - val_ssim: 0.7649 - val_acc: 0.9363

Epoch 00099: saving model to Bwt_rgrsn.hdf5
Epoch 100/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0574 - psnr: 13.6831 - ssim: 0.7885 - acc: 0.9428 - val_loss: 0.0614 - val_psnr: 12.9491 - val_ssim: 0.7741 - val_acc: 0.9388

Epoch 00100: saving model to Bwt_rgrsn.hdf5
Epoch 101/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0586 - psnr: 13.6022 - ssim: 0.7840 - acc: 0.9416 - val_loss: 0.0610 - val_psnr: 13.0098 - val_ssim: 0.7753 - val_acc: 0.9391

Epoch 00101: saving model to Bwt_rgrsn.hdf5
Epoch 102/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0577 - psnr: 13.7007 - ssim: 0.7875 - acc: 0.9425 - val_loss: 0.0606 - val_psnr: 13.0131 - val_ssim: 0.7771 - val_acc: 0.93


Epoch 00163: saving model to Bwt_rgrsn.hdf5
Epoch 164/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0395 - psnr: 15.7999 - ssim: 0.8525 - acc: 0.9607 - val_loss: 0.0559 - val_psnr: 13.6150 - val_ssim: 0.7943 - val_acc: 0.9443

Epoch 00164: saving model to Bwt_rgrsn.hdf5
Epoch 165/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0404 - psnr: 15.6241 - ssim: 0.8488 - acc: 0.9597 - val_loss: 0.0523 - val_psnr: 13.9150 - val_ssim: 0.8070 - val_acc: 0.9478

Epoch 00165: saving model to Bwt_rgrsn.hdf5
Epoch 166/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0391 - psnr: 15.8357 - ssim: 0.8539 - acc: 0.9611 - val_loss: 0.0547 - val_psnr: 13.6699 - val_ssim: 0.7985 - val_acc: 0.9454

Epoch 00166: saving model to Bwt_rgrsn.hdf5
Epoch 167/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0386 - psnr: 15.8726 - ssim: 0.8558 - acc: 0.9616 - val_loss: 0.0562 - val_psnr: 13.5209 - val_ssim: 0.7931 - val_acc: 0.9

...


Epoch 00943: saving model to Bwt_rgrsn.hdf5
Epoch 944/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0047 - psnr: 26.5881 - ssim: 0.9818 - acc: 0.9953 - val_loss: 0.0523 - val_psnr: 13.9074 - val_ssim: 0.8055 - val_acc: 0.9477

Epoch 00944: saving model to Bwt_rgrsn.hdf5
Epoch 945/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0047 - psnr: 26.4785 - ssim: 0.9819 - acc: 0.9953 - val_loss: 0.0507 - val_psnr: 14.0644 - val_ssim: 0.8114 - val_acc: 0.9493

Epoch 00945: saving model to Bwt_rgrsn.hdf5
Epoch 946/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0046 - psnr: 26.8358 - ssim: 0.9821 - acc: 0.9954 - val_loss: 0.0512 - val_psnr: 13.9557 - val_ssim: 0.8093 - val_acc: 0.9488

Epoch 00946: saving model to Bwt_rgrsn.hdf5
Epoch 947/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0047 - psnr: 26.4596 - ssim: 0.9815 - acc: 0.9953 - val_loss: 0.0505 - val_psnr: 14.0460 - val_ssim: 0.8126 - val_acc: 0.9


Epoch 01008: saving model to Bwt_rgrsn.hdf5
Epoch 1009/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0044 - psnr: 27.1008 - ssim: 0.9830 - acc: 0.9956 - val_loss: 0.0508 - val_psnr: 14.0013 - val_ssim: 0.8112 - val_acc: 0.9492

Epoch 01009: saving model to Bwt_rgrsn.hdf5
Epoch 1010/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0043 - psnr: 27.1525 - ssim: 0.9834 - acc: 0.9957 - val_loss: 0.0524 - val_psnr: 13.8619 - val_ssim: 0.8050 - val_acc: 0.9476

Epoch 01010: saving model to Bwt_rgrsn.hdf5
Epoch 1011/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0043 - psnr: 26.9602 - ssim: 0.9833 - acc: 0.9957 - val_loss: 0.0493 - val_psnr: 14.1690 - val_ssim: 0.8167 - val_acc: 0.9508

Epoch 01011: saving model to Bwt_rgrsn.hdf5
Epoch 1012/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0042 - psnr: 27.1008 - ssim: 0.9840 - acc: 0.9959 - val_loss: 0.0515 - val_psnr: 14.0271 - val_ssim: 0.8085 - val_acc:


Epoch 01073: saving model to Bwt_rgrsn.hdf5
Epoch 1074/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0038 - psnr: 27.7381 - ssim: 0.9856 - acc: 0.9962 - val_loss: 0.0486 - val_psnr: 14.2929 - val_ssim: 0.8193 - val_acc: 0.9514

Epoch 01074: saving model to Bwt_rgrsn.hdf5
Epoch 1075/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0038 - psnr: 27.7893 - ssim: 0.9853 - acc: 0.9962 - val_loss: 0.0537 - val_psnr: 13.7607 - val_ssim: 0.7997 - val_acc: 0.9463

Epoch 01075: saving model to Bwt_rgrsn.hdf5
Epoch 1076/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0044 - psnr: 26.5860 - ssim: 0.9828 - acc: 0.9956 - val_loss: 0.0521 - val_psnr: 13.9310 - val_ssim: 0.8062 - val_acc: 0.9479

Epoch 01076: saving model to Bwt_rgrsn.hdf5
Epoch 1077/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0044 - psnr: 27.1492 - ssim: 0.9830 - acc: 0.9957 - val_loss: 0.0506 - val_psnr: 14.0619 - val_ssim: 0.8121 - val_acc:


Epoch 01138: saving model to Bwt_rgrsn.hdf5
Epoch 1139/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0033 - psnr: 28.5078 - ssim: 0.9874 - acc: 0.9967 - val_loss: 0.0517 - val_psnr: 14.0236 - val_ssim: 0.8080 - val_acc: 0.9483

Epoch 01139: saving model to Bwt_rgrsn.hdf5
Epoch 1140/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0034 - psnr: 28.4850 - ssim: 0.9872 - acc: 0.9966 - val_loss: 0.0493 - val_psnr: 14.2594 - val_ssim: 0.8172 - val_acc: 0.9507

Epoch 01140: saving model to Bwt_rgrsn.hdf5
Epoch 1141/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0034 - psnr: 28.1604 - ssim: 0.9870 - acc: 0.9966 - val_loss: 0.0522 - val_psnr: 13.9387 - val_ssim: 0.8059 - val_acc: 0.9478

Epoch 01141: saving model to Bwt_rgrsn.hdf5
Epoch 1142/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0036 - psnr: 27.9195 - ssim: 0.9861 - acc: 0.9964 - val_loss: 0.0491 - val_psnr: 14.2941 - val_ssim: 0.8179 - val_acc:


Epoch 01203: saving model to Bwt_rgrsn.hdf5
Epoch 1204/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0037 - psnr: 27.7357 - ssim: 0.9859 - acc: 0.9963 - val_loss: 0.0528 - val_psnr: 14.0281 - val_ssim: 0.8044 - val_acc: 0.9472

Epoch 01204: saving model to Bwt_rgrsn.hdf5
Epoch 1205/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0040 - psnr: 26.8840 - ssim: 0.9841 - acc: 0.9960 - val_loss: 0.0497 - val_psnr: 14.2140 - val_ssim: 0.8157 - val_acc: 0.9503

Epoch 01205: saving model to Bwt_rgrsn.hdf5
Epoch 1206/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0040 - psnr: 27.1646 - ssim: 0.9845 - acc: 0.9961 - val_loss: 0.0527 - val_psnr: 13.8619 - val_ssim: 0.8039 - val_acc: 0.9473

Epoch 01206: saving model to Bwt_rgrsn.hdf5
Epoch 1207/70000
96/96 [==============================] - 7s 76ms/step - loss: 0.0037 - psnr: 27.6456 - ssim: 0.9855 - acc: 0.9963 - val_loss: 0.0515 - val_psnr: 14.1347 - val_ssim: 0.8095 - val_acc:

In [ ]:
#show_num = 10
#fig, ax = plt.subplots(show_num, 3, figsize=(15, 50))

#for i, pred in enumerate(preds[:show_num]):
   # ax[i, 0].imshow(x_test[i].squeeze(), cmap='gray')
   # ax[i, 1].imshow(y_test[i].squeeze(), cmap='gray')
   # ax[i, 2].imshow(pred.squeeze(), cmap='gray')

 ## Testing

In [ ]:
val=x_test # test set loading
val.shape

In [ ]:
model.load_weights('Bwt_rgrsn.hdf5')
per =  model.predict(val)

In [ ]:
i=5
 
q =  (val[i]) # corrupted code
 
q= q[:,:,0]  
q=q.astype('float32')
q=q/255
p=per[i] # prdiction upon the corrupted code
p= p[:,:,0] 
p=p.astype('float32')
org=y_test[i][:,:,0] # original code

## Corrupted vs raw prediction

In [ ]:
plot_sample( p,q)
 

## Original vs thresholded prediction

In [ ]:
# adaptive thresholding using image mean
mn=p.mean()
rm,pt = cv2.threshold(p,mn,1,cv2.THRESH_BINARY) 


In [ ]:
plot_sample( pt ,org)
 

## Mean absolute error comparasion

In [ ]:
from sklearn.metrics import mean_absolute_error,max_error

er1 = mean_absolute_error(org, p) # error for raw prediction
er2 = mean_absolute_error(org, pt) # error for thresholded prediction

print(er1)
print(er2)


## Listing the MAE between the original codes and the raw predicted codes

In [ ]:
err1=[]
rng=y_test.shape[0]
for i in range (rng):
    
    original=y_test[:,:,0][i]
    predict=per[:,:,0][i]
    maer= mean_absolute_error(original, predict)
    err1.append(maer)
err1=np.asarray(err1) 
 
#for i in range (rng):
    #print(err1[i])

## Listing the MAE between the original codes and the thresholded_predicted codes

In [ ]:
err2=[]
rng=y_test.shape[0]
for i in range (rng):
    
    original=y_test[:,:,0][i]
    predict=per[:,:,0][i]
    mn=predict.mean()
    rm,predict_th = cv2.threshold(predict,mn,1,cv2.THRESH_BINARY) 
    
    maer= mean_absolute_error(original, predict_th)
    
 
    err2.append(maer)
err2=np.asarray(err2) 
 
#for i in range (rng):
    #print(err2[i])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 


X = np.linspace(0, 2, 30)
error_prdicted = err1 
error_thresholded = err2

plt.plot(X, error_prdicted,label="error_prdicted")
plt.plot(X, error_thresholded ,label="error_thresholded")
plt.legend()
plt.show()
 

In [ ]:
rang = 10
fig, ax = plt.subplots(rang, 3, figsize=(15, 50))

for i, pred in enumerate(per[:rang]):
    ax[i, 0].imshow(x_test[i].squeeze(), cmap='gray')
    ax[i, 1].imshow(y_test[i].squeeze(), cmap='gray')
    ax[i, 2].imshow(pred.squeeze(), cmap='gray')